## SANDAG UrbanSim

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd, numpy as np
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import urbansim.sim.simulation as sim

### Run all models

In [ ]:
sim.run(["build_networks"]) #initialize network accessibility engine
sim.run(["scheduled_development_events", "neighborhood_vars", #scheduled events and accessibility variables
         "rsh_simulate", "nrh_simulate", "nrh_simulate2",   #price models
         "jobs_transition", "elcm_simulate", "households_transition", "hlcm_luz_simulate", #demand/location models
         "price_vars", "feasibility", "residential_developer", "non_residential_developer", #supply/proforma models
         "model_integration_indicators", #indicators for pasef/pecas
         "buildings_to_uc", #export buildings to urban canvas
         ], years=range(2013, 2016))

### LUZ Indicators

In [ ]:
bsim = sim.get_table('buildings').to_frame(columns = ['luz_id', 'note', 'res_price_per_sqft', 'nonres_rent_per_sqft', 'residential_units', 'non_residential_sqft'])
luz_res_price = bsim[bsim.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = bsim[bsim.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
bsim = bsim[(bsim.note == 'simulated') | bsim.note.str.startswith('Sitespec')]
luz_simdu = bsim.groupby('luz_id').residential_units.sum()
luz_simnr = bsim.groupby('luz_id').non_residential_sqft.sum()
luz_df = pd.DataFrame({'du':luz_simdu, 'nrsf':luz_simnr, 'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
luz_df = luz_df[luz_df.index.values != 0].fillna(0)
luz_df.index = luz_df.index.values.astype('int')
luz_df.index.name = 'luz_id'
from spandex import TableLoader; loader = TableLoader(); luz_df.to_csv(loader.get_path('out/luz_sim_indicators.csv'))

### MSA Indicators

In [ ]:
# Summarize results at MSA level
b = sim.get_table('buildings').to_frame(columns = ['msa_id', 'mgra_id', 'residential_units', 'non_residential_sqft', 'note'])
new_du_by_msa = b[b.note  == 'simulated'].groupby('msa_id').residential_units.sum()
new_nrsf_by_msa = b[b.note  == 'simulated'].groupby('msa_id').non_residential_sqft.sum()
proportion_du_by_msa = new_du_by_msa / new_du_by_msa.sum()
proportion_nrsf_by_msa = new_nrsf_by_msa / new_nrsf_by_msa.sum()
print proportion_du_by_msa
print proportion_nrsf_by_msa 

### Predict prices and summarize/export at the LUZ level

In [ ]:
%%capture
## Accessibility engine
sim.run(["build_networks", "neighborhood_vars"])

# Predict prices
sim.run(["rsh_simulate", "nrh_simulate", "nrh_simulate2"])

##Export LUZ mean prices to csv for visual checking
buildings = sim.get_table('buildings').to_frame()
luz_res_price = buildings[buildings.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = buildings[buildings.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
df = pd.DataFrame({'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
# df.fillna(0).to_csv(loader.get_path('out/sim/sd_prices.csv'))

# Summarize prices using network queries
sim.run(['price_vars'])

### Proforma

In [ ]:
# from urbansim.developer import sqftproforma
# # Step 1:  feasibility
sim.run(['feasibility'])
sim.run(["residential_developer"])
sim.run(["non_residential_developer"])